<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej"
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

Code based on https://github.com/pytorch/examples/blob/master/mnist/main.py

In this exercise we are using high level abstractions from torch.nn like nn.Linear.
Note: during the next lab session we will go one level deeper and implement more things
with bare hands.

Tasks:

    1. Read the code.

    2. Check that the given implementation reaches 95% test accuracy for architecture input-128-128-10 after few epochs.

    3. Add the option to use SGD with momentum instead of ADAM.

    4. Experiment with different learning rates. Use provided TrainingVisualizer
    to plot the learning curves and gradient to weight ratios. Compare visualizations
    for different learning rates for both ADAM and SGD with momentum.

    5. Parameterize the constructor by a list of sizes of hidden layers of the MLP.
    Note that this requires creating a list of layers as an atribute of the Net class,
    and one can't use a standard python list containing nn.Modules (why?).
    Check torch.nn.ModuleList.


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import plotly.graph_objects as go
import sys
if 'google.colab' in sys.modules:
    from google.colab import output
    output.enable_custom_widget_manager()

In [ ]:
# @title Visualize gradients

class TrainingVisualizer:
    def __init__(self, log_interval=10):
        self.log_interval = log_interval
        self.train_loss_fig = self.init_line_plot(title='Training loss', xaxis_title='Step')
        self.grad_to_weight_fig = self.init_line_plot(
            title='Gradient standard deviation to weight standard deviation ratio at 1st layer',
            xaxis_title='Step',
            yaxis_title='Gradient to weight ratio (log scale)',
            yaxis_type='log'
        )
        self.test_acc_fig = self.init_line_plot(
            title='Test accuracy', x=[], xaxis_title='Epoch', mode='lines+markers'
        )

        # Parameters related to current tracked model and its training
        self.first_linear_layer = None
        self.lr = None
        self.trace_idx = -1

    def init_line_plot(
        self,
        title,
        x=None, xaxis_title=None,
        yaxis_title=None, yaxis_type='linear',
        mode='lines'
    ):
        fig = go.Figure()
        fig.update_layout(
            title=title, title_x=0.5,
            xaxis_title=xaxis_title, yaxis_title=yaxis_title,
            height=400, width=1500, margin=dict(b=10, t=60)
        )
        fig.update_yaxes(type=yaxis_type)
        # We cannot add new traces dynamically because Colab has problem with widgets
        # from plotly (traces added dynamically are rendered twice).
        # As an ugly workaround we create a lot of empty traces and update them later
        # with actual data. Empty traces are not plotted.
        for _ in range(25):
            fig.add_trace(go.Scatter(x=x, y=[], showlegend=True, mode=mode))

        fig_widget = go.FigureWidget(fig)
        display(fig_widget)
        return fig_widget

    def track_model(self, model, optimizer, lr):
        """
        Start tracking training metrics for a new model.
        """

        for field in model.__dict__['_modules'].values():
            if isinstance(field, nn.Linear):
                self.first_linear_layer = field
                break
            elif isinstance(field, nn.ModuleList):
                self.first_linear_layer = field[0]
                break

        self.lr = lr
        self.trace_idx += 1

        optim_name = type(optimizer).__name__
        self.train_loss_fig.data[self.trace_idx].name = f'{optim_name}, {lr}'
        self.grad_to_weight_fig.data[self.trace_idx].name = f'{optim_name}, {lr}'
        self.test_acc_fig.data[self.trace_idx].name = f'{optim_name}, {lr}'

    def plot_gradients_and_loss(self, batch_idx, loss):
        if batch_idx % self.log_interval == 0:
            self.train_loss_fig.data[self.trace_idx].y += (loss, )

            layer = self.first_linear_layer
            grad_to_weight_ratio = (self.lr * layer.weight.grad.std() / layer.weight.std()).item()

            self.grad_to_weight_fig.data[self.trace_idx].y += (grad_to_weight_ratio, )

    def plot_accuracy(self, epoch, accuracy):
        self.test_acc_fig.data[self.trace_idx].x += (epoch, )
        self.test_acc_fig.data[self.trace_idx].y += (accuracy, )

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # After flattening an image of size 28x28 we have 784 inputs
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(model, device, train_loader, optimizer, epoch, log_interval, visualizer, verbose=False):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        visualizer.plot_gradients_and_loss(batch_idx, loss.item())
        optimizer.step()
        if batch_idx % log_interval == 0:
            if verbose:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader, visualizer, verbose=False):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    if verbose:
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))
    visualizer.plot_accuracy(epoch, 100. * correct / len(test_loader.dataset))



In [ ]:
batch_size = 256
test_batch_size = 1000
epochs = 5
lr = 1e-2
seed = 1
log_interval = 10
use_cuda = torch.cuda.is_available()

In [ ]:
torch.manual_seed(seed)
device = torch.device("cuda" if use_cuda else "cpu")

train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': test_batch_size}
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                    'pin_memory': True,
                    'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

In [ ]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                    transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                    transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
visualizer = TrainingVisualizer(log_interval=log_interval)

In [ ]:
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
visualizer.track_model(model, optimizer, lr)

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval, visualizer)
    test(model, device, test_loader, visualizer)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.313259
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.732760
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.582762
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.481087
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.334523
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.346265
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.229349
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.191099
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.224889
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.244065
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.236049
Train Epoch: 1 [28160/60000 (47%)]	Loss: 0.241033
Train Epoch: 1 [30720/60000 (51%)]	Loss: 0.194208
Train Epoch: 1 [33280/60000 (55%)]	Loss: 0.302403
Train Epoch: 1 [35840/60000 (60%)]	Loss: 0.203934
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.192445
Train Epoch: 1 [40960/60000 (68%)]	Loss: 0.173173
Train Epoch: 1 [43520/60000 (72%)]	Loss: 0.140804
Train Epoch: 1 [46080/60000 (77%)]	Loss: 0.356528
Train Epoch: 1 [48640/60000 (81%)]	Loss: 0.130329
Train Epoc